Способ 1 Как распарсить web-страницу 

In [4]:
import urllib3

In [5]:
from elasticsearch import Elasticsearch

In [6]:
client = Elasticsearch(hosts=["localhost:9200"])

In [7]:
# declare a function that will return a webpage's HTML as a string
def fetch_data(url):

    # catch any errors and print/return them
    try:
        # get the HTTP urllib3.connectionpool.HTTPSConnectionPool object
        http_pool = urllib3.connection_from_url(url)

        # returns a urllib3.response.HTTPResponse object
        resp = http_pool.urlopen('GET', url)

        # decode the HTML data using UTF-8
        decoded = resp.data.decode('utf-8')

    except Exception as err:
        print ("fetch_data() ERROR", err)
        decoded = "fetch_data() ERROR" + str(err)

    # return the urllib response or any errors
    return decoded

In [8]:
# iterate, slice, and parse HTML string data
def parse_html(data, marker, start, end):
    # iterate the list of HTML data
    for line in data:
        # make sure the HTML string contains marker
        if marker in line:
            line = line[line.find(marker):]

            # check if the start and end data is in line
            if (start in line) and (end in line):
                  # find where the target data starts and ends
                s = line.find(start) + len(start)
                e = line.find(end)

                # return the sliced string
                return line[s : e]

    # return empty string by default
    return ""
        

In [17]:
# declare a URL string for the urllib to parse
url = "http://localhost/TOIR/odata/standard.odata/Catalog_%D1%82%D0%BE%D1%80%D0%BE_%D0%9E%D0%B1%D1%8A%D0%B5%D0%BA%D1%82%D1%8B%D0%A0%D0%B5%D0%BC%D0%BE%D0%BD%D1%82%D0%B0?$format=json&$filter=Description%20eq%20%27%D0%97%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5%20%D1%86%D0%B5%D1%85%D0%B0%20%E2%84%961%27&$select=%D0%98%D0%BD%D0%B2%D0%B5%D0%BD%D1%82%D0%B0%D1%80%D0%BD%D1%8B%D0%B9%D0%9D%D0%BE%D0%BC%D0%B5%D1%80"

# pass the URL string to the fetch_data() function
source = fetch_data(url)

In [18]:
# use Python's split() method to put lines of HTML into a list []
source = source.split("\n") # "\n" == linebreak

# print the number of HTML lines returned by urllib
print ("Lines to iterate:", len(source))

Lines to iterate: 6



Способ 2 Как распарсить web-страницу 

In [14]:
import pandas as pd

In [15]:
data=pd.read_html(url)

ValueError: No tables found

Способ 3 Как распарсить web-страницу

In [22]:
import requests
res = requests.get('http://localhost:9200')
print(res.content)

b'{\n  "name" : "IL706",\n  "cluster_name" : "elasticsearch",\n  "cluster_uuid" : "uffXvGunTLqXUVxckq9hhQ",\n  "version" : {\n    "number" : "7.9.1",\n    "build_flavor" : "default",\n    "build_type" : "zip",\n    "build_hash" : "083627f112ba94dffc1232e8b42b73492789ef91",\n    "build_date" : "2020-09-01T21:22:21.964974Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.6.2",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [23]:
es=Elasticsearch()

In [31]:
es.index(index='sw', doc_type='people', id=1, body={
	"name": "Luke Skywalker",
	"height": "172",
	"mass": "77",
	"hair_color": "blond",
	"birth_year": "19BBY",
	"gender": "male",
})

{'_index': 'sw',
 '_type': 'people',
 '_id': '1',
 '_version': 3,
 'result': 'updated',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 2,
 '_primary_term': 1}

In [51]:
import json
r = requests.get('http://localhost:9200')
i = 1
r = requests.get('http://localhost:9200/sw/people/'+ str(i))
respons=r.json()

print(respons)

{'_index': 'sw', '_type': 'people', '_id': '1', '_version': 3, '_seq_no': 2, '_primary_term': 1, 'found': True, '_source': {'name': 'Luke Skywalker', 'height': '172', 'mass': '77', 'hair_color': 'blond', 'birth_year': '19BBY', 'gender': 'male'}}


Считываем с сайта, где есть API, и вносим в индекс ES

In [52]:
import json
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get('http://swapi.dev/api/people/'+ str(i)+'/')
    es.index(index='sw', doc_type='people', id=i, body=json.loads(r.content))
    i=i+1
    
print(i)

18


In [57]:
r = requests.get('http://localhost:9200/sw/people/'+ str(5))
respons=r.json()

print(respons)

{'_index': 'sw', '_type': 'people', '_id': '5', '_version': 1, '_seq_no': 7, '_primary_term': 1, 'found': True, '_source': {'name': 'Leia Organa', 'height': '150', 'mass': '49', 'hair_color': 'brown', 'skin_color': 'light', 'eye_color': 'brown', 'birth_year': '19BBY', 'gender': 'female', 'homeworld': 'http://swapi.dev/api/planets/2/', 'films': ['http://swapi.dev/api/films/1/', 'http://swapi.dev/api/films/2/', 'http://swapi.dev/api/films/3/', 'http://swapi.dev/api/films/6/'], 'species': [], 'vehicles': ['http://swapi.dev/api/vehicles/30/'], 'starships': [], 'created': '2014-12-10T15:20:09.791000Z', 'edited': '2014-12-20T21:17:50.315000Z', 'url': 'http://swapi.dev/api/people/5/'}}


In [69]:
es.search(index="sw", body={"query": {"prefix":{ "name" : "Leia Organa" }}})

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [74]:
es.get(index='sw', doc_type='people', id=5)

{'_index': 'sw',
 '_type': 'people',
 '_id': '5',
 '_version': 1,
 '_seq_no': 7,
 '_primary_term': 1,
 'found': True,
 '_source': {'name': 'Leia Organa',
  'height': '150',
  'mass': '49',
  'hair_color': 'brown',
  'skin_color': 'light',
  'eye_color': 'brown',
  'birth_year': '19BBY',
  'gender': 'female',
  'homeworld': 'http://swapi.dev/api/planets/2/',
  'films': ['http://swapi.dev/api/films/1/',
   'http://swapi.dev/api/films/2/',
   'http://swapi.dev/api/films/3/',
   'http://swapi.dev/api/films/6/'],
  'species': [],
  'vehicles': ['http://swapi.dev/api/vehicles/30/'],
  'starships': [],
  'created': '2014-12-10T15:20:09.791000Z',
  'edited': '2014-12-20T21:17:50.315000Z',
  'url': 'http://swapi.dev/api/people/5/'}}

In [72]:
es.search(index="sw", body={"query": {"match": {'name':'Darth Vader'}}})

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 4.9164443,
  'hits': [{'_index': 'sw',
    '_type': 'people',
    '_id': '4',
    '_score': 4.9164443,
    '_source': {'name': 'Darth Vader',
     'height': '202',
     'mass': '136',
     'hair_color': 'none',
     'skin_color': 'white',
     'eye_color': 'yellow',
     'birth_year': '41.9BBY',
     'gender': 'male',
     'homeworld': 'http://swapi.dev/api/planets/1/',
     'films': ['http://swapi.dev/api/films/1/',
      'http://swapi.dev/api/films/2/',
      'http://swapi.dev/api/films/3/',
      'http://swapi.dev/api/films/6/'],
     'species': [],
     'vehicles': [],
     'starships': ['http://swapi.dev/api/starships/13/'],
     'created': '2014-12-10T15:18:20.704000Z',
     'edited': '2014-12-20T21:17:50.313000Z',
     'url': 'http://swapi.dev/api/people/4/'}}]}}